In [1]:
from influxdb import *
import pandas as pd
from pathlib import Path
import os
import json
from datetime import datetime

In [2]:
client = InfluxDBClient(host='influx.sorinmircea.com', database="lifelogger_test", port=443, username='root', password='root', ssl=True,verify_ssl=True)
client.ping()

'1.7.10'

In [3]:
client.get_list_database()

[{'name': '_internal'},
 {'name': 'lifelogger_test'},
 {'name': 'lifelogger_samsung'}]

In [4]:
client.drop_database("lifelogger_samsung")
client.create_database("lifelogger_samsung")
client.switch_database("lifelogger_samsung")

In [5]:
# ----- HEART RATE DATA ----------
path = "data/samsung/jsons/com.samsung.shealth.tracker.heart_rate"
bpm_files = Path(path).glob("**/*.json")
for file in bpm_files:
    fileOpened = open(file)
    bpm_entries = json.load(fileOpened)

    grafana_bpm_entries = []
    for bpm_entry in bpm_entries:
        grafana_bpm_entries.append({
            "measurement": "heart_rate",
            "tags": {
            },
            "time": datetime.fromtimestamp(bpm_entry["start_time"]/1000.0).strftime('%Y-%m-%dT%H:%M:%SZ'),
            "fields": {
                "value": bpm_entry["heart_rate"]
            }
        })
    client.write_points(grafana_bpm_entries)
print(f"Done")


Done


In [6]:
# ----- STRESS RATE DATA ----------
path = "data/samsung/com.samsung.shealth.stress"
stress_files = Path(path).glob("**/*.json")
for file in stress_files:
    fileOpened = open(file)
    bpm_entries = json.load(fileOpened)

    grafana_stress_entries = []
    for stress_entry in bpm_entries:
        grafana_stress_entries.append({
            "measurement": "stress_rate",
            "tags": {
            },
            "time": datetime.fromtimestamp(stress_entry["start_time"]/1000.0).strftime('%Y-%m-%dT%H:%M:%SZ'),
            "fields": {
                "value": stress_entry["score"]
            }
        })
    client.write_points(grafana_stress_entries)
print(f"Done")

Done


In [7]:
# ----- ACTIVITY DAY SUMMARY DATA ----------
summary_table = pd.read_csv('data/samsung/com.samsung.shealth.activity.day_summary.202102121121.csv', index_col=False, skiprows=1)
summary_table = summary_table[summary_table['active_time'] != 0]

step_count = []
calories = []
distance = []
score = []
active_time = []
run_time = []
walk_time = []
longest_idle_time = []
def getInfluxFormat(measurement, row, value):
    return {
        "measurement": measurement,
        "tags": {
        },
        "time": row["update_time"],
        "fields": {
            "value": value
        }
    }
for index, row in summary_table.iterrows():
    step_count.append(getInfluxFormat("step_count", row, int(row["step_count"])))
    calories.append(getInfluxFormat("calorie", row, int(row["calorie"])))
    distance.append(getInfluxFormat("distance", row, int(row["distance"])))
    score.append(getInfluxFormat("score", row, row["score"]))
    active_time.append(getInfluxFormat("active_time", row, int(row["active_time"] / 60000)))
    run_time.append(getInfluxFormat("run_time", row, int(row["run_time"] / 60000)))
    walk_time.append(getInfluxFormat("walk_time", row, int(row["walk_time"] / 60000)))
    longest_idle_time.append(getInfluxFormat("longest_idle_time", row, int(row["longest_idle_time"] / 60000)))

client.write_points(step_count)
client.write_points(calories)
client.write_points(distance)
client.write_points(score)
client.write_points(active_time)
client.write_points(run_time)
client.write_points(walk_time)
client.write_points(longest_idle_time)

FileNotFoundError: [Errno 2] No such file or directory: 'data/com.samsung.shealth.activity.day_summary.202102121121.csv'